# Shared Structure

In [ ]:
(setf part (list 'b 'c))
(setf whole (cons 'a part))

; tailp
(tailp part whole)

(defun our-tailp (x y)
  (or (eql x y)
      (and (consp y)
           (our-tailp x (cdr y)))))
(our-tailp part whole)

(B C)

(A B C)

T

OUR-TAILP

T

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PART
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PART
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::WHOLE
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PART
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::WHOLE
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::PART
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::WHOLE


In [ ]:
(defun our-copy-list (lst)
  (if (null lst)
      nil
      (cons (car lst) (our-copy-list (cdr lst)))))

(defun our-copy-tree (tr)
  (if (atom tr)
      tr
      (cons (our-copy-tree (car tr))
            (our-copy-tree (cdr tr)))))

OUR-COPY-LIST

OUR-COPY-TREE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::OUR-COPY-LIST in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::OUR-COPY-TREE in DEFUN


# Modification

In [ ]:
(setf whole (list 'a 'b 'c)
  tail (cdr whole))

(setf (second tail) 'e)
tail
whole

(B C)

E

(B E)

(A B E)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TAIL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::WHOLE
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::WHOLE
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TAIL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TAIL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::WHOLE


# Example: Queues

In [ ]:
(defun make-queue ()
  (cons nil nil)) ; head, tail

(defun enqueue (obj q)
  (if (null (car q))
      (setf (cdr q) (setf (car q) (list obj)))
      (setf (cdr (cdr q)) (list obj)
        (cdr q) (cdr (cdr q))))
  (car q))

(defun dequeue (q)
  (pop (car q)))

MAKE-QUEUE

ENQUEUE

DEQUEUE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-QUEUE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ENQUEUE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::DEQUEUE in DEFUN


In [ ]:
(let ((q1 (make-queue)))
  (enqueue 'a q1)
  (enqueue 'b q1)
  (enqueue 'c q1)
  (print q1)
  (dequeue q1)
  (dequeue q1)
  (enqueue 'd q1)
  (print q1))

((C D) D)


((A B C) C) 
((C D) D) 

# Destructive Functions

In [1]:
; delete: destructive version of remove
(let ((lst '(a r a b i a)))
  (print (delete 'a lst))
  (print lst))

(A R B I)

SB-INT:CONSTANT-MODIFIED: Destructive function DELETE called on constant data: (A R A B I A)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1

(R B I) 
(A R B I) 

In [1]:
; nconc: destructive version of append
(defun nconc2 (x y)
  (if (consp x)
      (progn
       (setf (cdr (last x)) y)
       x)
      y))

(defun our-mapcan (fn &rest lsts)
  (apply #'nconc (apply #'mapcar fn lsts)))

(our-mapcan #'list '(a b c) '(1 2 3 4))

NCONC2

OUR-MAPCAN

(A 1 B 2 C 3)

# Example: Binary Search Trees

In [2]:

;;; Node
(defstruct (node (:print-function
                  (lambda (n s d)
                    (format s "#<~A>" (node-elt n)))))
  ;; element
  elt
  ;; left child
  (l nil)
  ;; right child
  (r nil))

(defun bst-insert (obj bst <)
  (if (null bst)
      (make-node :elt obj)
      (let ((elt (node-elt bst)))
        (if (eql obj elt)
            bst
            (if (funcall < obj elt)
                ;; true: insert left
                (make-node :elt elt
                           :l (bst-insert obj (node-l bst) <)
                           :r (node-r bst))
                (make-node :elt elt
                           :r (bst-insert obj (node-r bst) <)
                           :l (node-l bst)))))))

(defun bst-find (obj bst <)
  (if (null bst)
      nil
      (let ((elt (node-elt bst)))
        (if (eql obj elt)
            bst
            (if (funcall < obj elt)
                ;; true: find in left
                (bst-find obj (node-l bst) <)
                (bst-find obj (node-r bst) <))))))

(defun bst-min (bst)
  (and bst
       (or (bst-min (node-l bst)) bst)))

(defun bst-max (bst)
  (and bst
       (or (bst-max (node-r bst)) bst)))

(defun bst-remove (obj bst <)
  (if (null bst)
      nil
      (let ((elt (node-elt bst)))
        (if (eql obj elt)
            (percolate bst) ; found
            (if (funcall < obj elt)
                ;; true: delete in left
                (make-node :elt elt
                           :l (bst-remove obj (node-l bst) <)
                           :r (node-r bst))
                (make-node :elt elt
                           :r (bst-remove obj (node-r bst) <)
                           :l (node-l bst)))))))

(defun percolate (bst)
  (cond ((null (node-l bst)) ; left null
                            (if (null (node-r bst))
                                nil
                                (rperc bst)))
        ((null (node-r bst)) (lperc bst)) ; right nul
        (t (if (zerop (random 2)) ; both not null
               (lperc bst)
               (rperc bst)))))

(defun rperc (bst)
  (make-node :elt (node-elt (node-r bst))
             :l (node-l bst)
             :r (percolate (node-r bst))))

(defun lperc (bst)
  (make-node :elt (node-elt (node-l bst))
             :l (percolate (node-l bst))
             :r (node-r bst)))


(defun bst-traverse (fn bst)
  (when bst
        (bst-traverse fn (node-l bst))
        (funcall fn (node-elt bst))
        (bst-traverse fn (node-r bst))))

NODE

BST-INSERT

BST-FIND

BST-MIN

BST-MAX

...

PERCOLATE

RPERC

LPERC

BST-TRAVERSE

SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer COMMON-LISP-USER::NODE when executing
SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a STANDARD-METHOD of a
STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: The variable D is defined but never used.
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::NODE-ELT
SB-C:INLINING-DEPENDENCY-FAILURE: Previously compiled call to COMMON-LISP-USER::NODE-ELT could not be inlined
because the structure definition for COMMON-LISP-USER::NODE was not yet seen.
To avoid this warning, DEFSTRUCT should precede references to the affected
functions, or they must be declared locally notinline at each call site.
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::PERCOLATE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::LPERC
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::RPERC


In [3]:
(let ((nums nil))
  (dolist (x '(5 8 4 2 1 9 6 7 3))
    (setf nums (bst-insert x nums #'<)))
  (print nums)
  (bst-traverse #'princ nums)
  (print (list
          (bst-find 12 nums #'<)
          (bst-find 4 nums #'<)
          (bst-min nums)
          (bst-max nums)))
  (setf nums (bst-remove 2 nums #'<))
  (print (bst-find 2 nums #'<))
  (bst-traverse #'princ nums))

NIL


#<5> 123456789
(NIL #<4> #<1> #<9>) 
NIL 13456789

# Example: Doubly-Linked Lists

In [2]:
(defstruct (dl (:print-function print-dl))
  prev
  data
  next)

(defun print-dl (dl stream depth)
  (declare (ignore depth))
  (format stream "#<DL ~A>" (dl->list dl)))

(defun dl->list (lst)
  (if (dl-p lst)
      (cons (dl-data lst) (dl->list (dl-next lst)))
      lst))

(defun dl-insert (x lst)
  (let ((elt (make-dl :data x :next lst)))
    (when (dl-p lst)
          (if (dl-prev lst)
              (setf (dl-next (dl-prev lst)) elt
                (dl-prev elt) (dl-prev lst)))
          (setf (dl-prev lst) elt))
    elt))

(defun dl-list (&rest args)
  (reduce #'dl-insert args
    :from-end t :initial-value nil))

(defun dl-remove (lst)
    (if (dl-prev lst)
        (setf (dl-next (dl-prev lst)) (dl-next lst)))
    (if (dl-next lst)
        (setf (dl-prev (dl-next lst)) (dl-prev lst)))
    (dl-next lst))

DL

PRINT-DL

DL->LIST

DL-INSERT

DL-LIST

DL-REMOVE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COPY-DL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::DL-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::DL-PREV) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::DL-PREV in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::DL-DATA) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::DL-DATA in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::DL-NEXT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::DL-NEXT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-DL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining PRINT-OBJECT (#<STRUCTURE-CLASS COMMON-LISP-USER::DL> #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER

In [3]:
(setf dl (dl-list 'a 'b))
(setf dl (dl-insert 'c dl))
(dl-insert 'r (dl-next dl))
dl

#<DL (A B)>

#<DL (C A B)>

#<DL (R A B)>

#<DL (C R A B)>

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::DL


# Circular Structure

In [4]:
; cdr-circular
(setf x (list 'a))
(progn (setf (cdr x) x) nil)
(setf *print-circle* t)
x

(A)

NIL

T

#1=(A . #1#)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X


In [5]:
(defun circular (lst)
    (setf (cdr (last lst)) lst))

CIRCULAR

In [6]:
; car-circular
(let ((y (list 'a)))
    (setf (car y) y)
    y)

#1=(#1#)

In [7]:
; both
(let ((c (cons 1 1)))
  (setf (car c) c
    (cdr c) c)
  c)

#1=(#1# . #1#)

In [8]:
; for array
(setf *print-array* t)
(let ((a (make-array 1)))
    (setf (aref a 0) a)
    a)

T

#1=#(#1#)

In [10]:
; for struct
(progn (defstruct elt2
         (parent nil) (child nil))
       (let ((c (make-elt2))
             (p (make-elt2)))
         (setf (elt2-parent c) p
           (elt2-child p) c)
         c))

#1=#S(ELT2 :PARENT #S(ELT2 :PARENT NIL :CHILD #1#) :CHILD NIL)

SB-INT:SIMPLE-STYLE-WARNING: undefined function: (COMMON-LISP:SETF COMMON-LISP-USER::ELT2-CHILD)
SB-INT:SIMPLE-STYLE-WARNING: undefined function: (COMMON-LISP:SETF COMMON-LISP-USER::ELT2-PARENT)
SB-C:INLINING-DEPENDENCY-FAILURE: Previously compiled calls to (COMMON-LISP:SETF COMMON-LISP-USER::ELT2-PARENT)
and (COMMON-LISP:SETF COMMON-LISP-USER::ELT2-CHILD) could not be inlined
because the structure definition for COMMON-LISP-USER::ELT2 was not yet seen.
To avoid this warning, DEFSTRUCT should precede references to the affected
functions, or they must be declared locally notinline at each call site.


# Constatnt Structure

In [1]:
(defun arith-op (x)
    (member x '(+ - * /)))

ARITH-OP

In [2]:
(let ()
    (nconc (arith-op '*) '(as it were))
    (arith-op 'as))

NIL

In [3]:
(defun arith-op2 (x)
    (find x '(+ - * /)))

ARITH-OP2

In [5]:
(let ()
    (arith-op2 '*))

*